In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from src.constants import *

from src.utility.personal import *
from src.utility.structure import *
from src.utility.storage import *
from src.utility.math import *

from src.avgs import *
from src.trajectory import *

from src.rolling_data import RollingData
from src.per_game_model import PerGameModel
from src.init_DFs.per_game import PerGameInit
from src.init_DFs.next_game import NextGameInit

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

pgModel = PerGameModel()
initPG = PerGameInit()
initNextGame = NextGameInit()

In [2]:
season = "2022"
situation = "5on5"
df = primary_csv(season, situation)

num_games = 8

In [4]:
df_B = trajectory_linear(
    df.copy(),
    num_games,
    add_objs=["season", "game_number", "is_home", "iceTime"],
    suffix=False
)

b1=df_B[1]
b0=df_B[0]

export_csv(b0.copy(), season, situation, 
           name=f'LINEAR_{num_games}_b0', subfol="TRAJECTORY", 
           pretty=True)
export_csv(b1.copy(), season, situation, 
           name=f'LINEAR_{num_games}_b1', subfol="TRAJECTORY", 
           pretty=True)

In [14]:
b1=df_B[1]
b0=df_B[0]
b1

,game_id,team,opp_team,season,game_number,next_game_id,score,opp_score,win,reg_win,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
0,2014-11-08BUFPIT,BUF,PIT,2014,16,2014-11-11STLBUF,0.000000,-0.020588,0.001471,0.001471,...,0.258228,0.252941,0.214125,-0.014706,0.001643,-0.000015,0.000153,0.024662,0.020690,0.019722
1,2014-11-08FLACGY,CGY,FLA,2014,16,2014-11-10CARCGY,0.125000,0.060294,0.016176,0.023529,...,0.238737,0.133824,0.146476,-0.073529,0.004669,0.021479,0.020641,0.027079,0.028147,0.027210
2,2014-11-08PHICOL,COL,PHI,2014,16,2014-11-11NYICOL,0.141176,-0.005882,0.008824,-0.010294,...,0.539618,0.310294,0.288097,0.054412,0.004428,0.003500,0.003500,0.037563,0.033703,0.033735
3,2014-11-09ANAVAN,ANA,VAN,2014,16,2014-11-12ANALAK,-0.186765,-0.035294,-0.044118,-0.041176,...,0.447662,0.305882,0.142724,0.138235,0.001956,-0.002257,0.000840,-0.009991,-0.019982,-0.020471
4,2014-11-09ANAVAN,VAN,ANA,2014,16,2014-11-11VANOTT,-0.092647,-0.044118,-0.001471,-0.008824,...,-0.135826,-0.066176,-0.056934,0.250000,-0.002154,-0.010165,-0.010165,-0.002668,-0.000128,0.002066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17371,2023-04-13WSHNJD,WSH,NJD,2022,82,NaN,-0.105882,0.052941,-0.026471,-0.010294,...,-0.281219,-0.186765,-0.112012,0.077941,-0.003432,0.004268,0.004268,0.000975,0.003810,0.002087
17372,2023-04-14CBJBUF,BUF,CBJ,2022,82,NaN,0.114706,-0.160294,0.054412,0.016176,...,0.774912,0.626471,0.694760,-0.125000,0.007665,0.003162,0.003162,0.062991,0.068540,0.065537
17373,2023-04-14CBJBUF,CBJ,BUF,2022,82,NaN,-0.107353,-0.142647,0.000000,0.000000,...,-0.528922,-0.188235,-0.242284,0.058824,0.000491,0.001181,0.001181,0.017969,0.010096,0.010807
17374,2023-04-14NSHCOL,COL,NSH,2022,82,NaN,-0.041176,0.035294,-0.001471,-0.011765,...,-0.231497,-0.080882,-0.085072,0.167647,-0.002068,0.011559,0.011559,-0.019347,-0.018551,-0.018850


In [15]:
export_csv(b0.copy(), season, situation, 
           name=f'LINEAR_{num_games}_b0', subfol="TRAJECTORY", 
           pretty=True)
export_csv(b1.copy(), season, situation, 
           name=f'LINEAR_{num_games}_b1', subfol="TRAJECTORY", 
           pretty=True)